In [ ]:
from dotenv import load_dotenv
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    collection_name="constitucion_argentina_v2",
    persist_directory="vector_store",
    embedding_function=embedding
)

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """
Eres un abogado especializado en derecho constitucional argentino y debes utilizar la Constitución Argentina y la jurisprudencia relevante para responder preguntas y resolver dudas relacionadas con derechos y obligaciones legales de los ciudadanos argentinos. Deberá proporcionar información y asesoramiento profesional sobre si los derechos de un individuo están siendo infringidos y si existen fundamentos para iniciar acciones legales en distintas situaciones. El agente debe ofrecer respuestas claras, precisas y fundamentadas, al nivel de un profesional experimentado en el ámbito del derecho.
La respuesta debe estar amparada en alguno de los articulos de la constitucion que se detallan a continuacion:
{articulos_de_la_constitucion}
Reponde la siguiente pregunta con total seguridad y fundamentos en menos de 4 parrafos y sin sugerir consultas a otros profesionales:
{pregunta}
"""

In [ ]:
from langchain.schema import StrOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnableMap

prompt = ChatPromptTemplate.from_template(template=template)
retriever = vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10)
model = ChatOpenAI(temperature=0.2)
output_parser = StrOutputParser()

chain = RunnableMap({
    "articulos_de_la_constitucion": lambda x: retriever.get_relevant_documents(x["pregunta"]),
    "pregunta": lambda x: x["pregunta"]
}) | prompt | model | output_parser

In [ ]:
respuesta = chain.invoke({
    "pregunta": "¿Puedo salir de mi casa?"
})

In [ ]:
print(respuesta)

In [ ]:
respuesta = chain.invoke({
    "pregunta": "¿Puedo cortar la calle para realizar una manifestacion ya que mi empleador no me quiere subir el salario?"
})

In [ ]:
print(respuesta)

In [ ]:
respuesta = chain.invoke({
    "pregunta": "¿A que hace referencia el articulo 18 sobre el principio de legalidad?"
})

In [ ]:
print(respuesta)

In [ ]:
respuesta = chain.invoke({
    "pregunta": "¿Que pasa con los tratados internacionales firmados por Argentina, tiene algun rango de jerarquía frente a la constitucion?"
})

In [ ]:
print(respuesta)

In [ ]:
respuesta = chain.invoke({
    "pregunta": "¿A que hace referecia el articulo 14 bis sobre el derecho de los trabajadores?"
})